Import stuff

In [1]:
import xarray as xr
import pickle as pk
import time
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib as mpl
import mapclassify as mc
from copy import deepcopy as cp
import os
import matplotlib.pyplot as plt
import pandas as pd
import dask

In [2]:
# extreme event
global flags

flags = {}
flags['obs'] = 0           # 0: do not process ISIMIP3a obs tasmax runs (i.e. load runs pickle)
                           # 1: process ISIMIP runs (i.e. produce and save runs as pickle)
flags['pic'] = 0           # 0: do not process ISIMIP2b pic tasmax runs (i.e. load runs pickle)
                           # 1: process ISIMIP runs (i.e. produce and save runs as pickle)


{'obs': 0, 'pic': 0}


In [3]:
print(os.getcwd())

/vscmnt/brussel_pixiu_data/_data_brussel/vo/000/bvo00012/vsc10116/lifetime_exposure_isimip/hwmid


In [6]:
# import my own settings/functions/
from settings import *
from load_manip import *

In [7]:
# set global variables
init()

Read in PIC data from GCMs

In [9]:
ds_pic = collect_arrays(
    gcms,
    pdir
)

In [10]:
# check datasets
ds_pic

,Array,Chunk
Bytes,155.07 GiB,171.88 kiB
Shape,"(160600, 360, 720)","(440, 10, 10)"
Count,10606567 Tasks,1026432 Chunks
Type,float32,numpy.ndarray


In [11]:
ds = ds_pic
da = "MIROC5"

In [12]:
da_max = ds['tasmax_{}'.format(da)].groupby('time.year').max('time').chunk({'year':-1})

In [13]:
da_max

,Array,Chunk
Bytes,435.06 MiB,171.88 kiB
Shape,"(440, 360, 720)","(440, 10, 10)"
Count,21329671 Tasks,2592 Chunks
Type,float32,numpy.ndarray


In [14]:
ds[da+'_75'] = da_max.quantile(
    q=0.75,
    dim='year',
    method='inverted_cdf',
)

ds[da+'_25'] = da_max.quantile(
    q=0.25,
    dim='year',
    method='inverted_cdf',
)

In [15]:
ds

<xarray.Dataset>
Dimensions:        (time: 160600, lat: 360, lon: 720)
Coordinates:
  * lon            (lon) float32 -179.8 -179.2 -178.8 ... 178.8 179.2 179.8
  * lat            (lat) float32 89.75 89.25 88.75 ... -88.75 -89.25 -89.75
  * time           (time) object 1661-01-01 00:00:00 ... 2100-12-31 00:00:00
    quantile       float64 0.75
Data variables:
    tasmax_MIROC5  (time, lat, lon) float32 dask.array<chunksize=(440, 10, 10), meta=np.ndarray>
    MIROC5_75      (lat, lon) float64 dask.array<chunksize=(10, 10), meta=np.ndarray>
    MIROC5_25      (lat, lon) float64 dask.array<chunksize=(10, 10), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    title:        CMIP5 output of tasmax_day_MIROC5_piControl_r1i1p1, first-o...
    institution:  Potsdam Institute for Climate Impact Research, Research Dom...
    project:      Inter-Sectoral Impact Model Intercomparison Project (ISIMIP)
    contact:      Stefan Lange (slange@pik-potsdam.de)
    references:   Bias correction method: Hempel et al. (2013) Earth System D...

In [16]:
start_time = time.time()
ds[da+'_90'] = xr.ones_like(ds['tasmax_{}'.format(da)]).isel(time=slice(0,365)).groupby('time.dayofyear').mean('time')
print("--- {} minutes ---".format(
    np.floor((time.time() - start_time) / 60),
    )
      )

--- 0.0 minutes ---


In [17]:
ds

<xarray.Dataset>
Dimensions:        (time: 160600, lat: 360, lon: 720, dayofyear: 365)
Coordinates:
  * lon            (lon) float32 -179.8 -179.2 -178.8 ... 178.8 179.2 179.8
  * lat            (lat) float32 89.75 89.25 88.75 ... -88.75 -89.25 -89.75
  * time           (time) object 1661-01-01 00:00:00 ... 2100-12-31 00:00:00
    quantile       float64 0.75
  * dayofyear      (dayofyear) int64 1 2 3 4 5 6 7 ... 360 361 362 363 364 365
Data variables:
    tasmax_MIROC5  (time, lat, lon) float32 dask.array<chunksize=(440, 10, 10), meta=np.ndarray>
    MIROC5_75      (lat, lon) float64 dask.array<chunksize=(10, 10), meta=np.ndarray>
    MIROC5_25      (lat, lon) float64 dask.array<chunksize=(10, 10), meta=np.ndarray>
    MIROC5_90      (dayofyear, lat, lon) float32 dask.array<chunksize=(1, 10, 10), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    title:        CMIP5 output of tasmax_day_MIROC5_piControl_r1i1p1, first-o...
    institution:  Potsdam Institute for Climate Impact Research, Research Dom...
    project:      Inter-Sectoral Impact Model Intercomparison Project (ISIMIP)
    contact:      Stefan Lange (slange@pik-potsdam.de)
    references:   Bias correction method: Hempel et al. (2013) Earth System D...

In [18]:
# run window stats on each day
# for d in np.arange(1,366):
d = 50

i = d-15 # window start
f = d+15 # window end
w = np.arange(i,f+1) # window in calendar integers

if np.any(w<0): # incase negative values

    w = np.where(
        w>0,
        w,
        365+w,
    )

elif np.any(w>365): # incase values over 365

    w = np.where(
        w<=365,
        w,
        w-365,
    )                

In [19]:
w

array([35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65])

In [ ]:
# per calendar day, d, assign 90% quantile from original array using window; need to unchunk time after selecting 31 day window
start_time = time.time()
ds[da+'_90'].loc[
    {'dayofyear':d}
] = ds['tasmax_{}'.format(da)].sel(time=ds['time.dayofyear'].isin(w)).chunk({'time':-1}).quantile(
        q=0.9,
        dim='time',
        method='inverted_cdf',
    )
print("--- {} minutes ---".format(
    np.floor((time.time() - start_time) / 60),
    )
      )

In [ ]:
print(ds)

Get HWMId PIC quantiles

In [ ]:
start_time = time.time()
ds_pic = hwmid_qntls(
    ds_pic
)
print("--- {} minutes ---".format(
    np.floor((time.time() - start_time) / 60),
    )
      )

In [ ]:
# add quantiles for hwmid calc
os.chdir(cdir)
if not os.path.isfile('./data/pickles/hwmid_90.pkl'):
    ds_pic = hwmid_qntls(
        ds_pic
    )
    da_90 = ds_pic['MIROC5_90']
    with open('./data/pickles/hwmid_90.pkl','wb') as f:
        pk.dump(da_90,f)
    da_25 = ds_pic['MIROC5_25']
    with open('./data/pickles/hwmid_25.pkl','wb') as f:
        pk.dump(da_25,f)
    da_75 = ds_pic['MIROC5_75']
    with open('./data/pickles/hwmid_75.pkl','wb') as f:
        pk.dump(da_75,f)
    
else:
    with open('./data/pickles/hwmid_90.pkl','rb') as f:
        da_90 = pk.load(f)
    with open('./data/pickles/hwmid_75.pkl','rb') as f:
        da_75 = pk.load(f)        
    with open('./data/pickles/hwmid_25.pkl','rb') as f:
        da_25 = pk.load(f)  

In [ ]:
ds_pic['MIROC5'].time


Testing hot period here below

In [ ]:
# testing hot_period function
da_t = ds_pic['MIROC5']
da_t
da_90_cmp = xr.concat(
    [
        da_90.assign_coords({'dayofyear':pd.date_range(
                str(y),
                periods=da_90.sizes['dayofyear'],
                freq='D'
            )}).rename({'dayofyear':'time'}) for y in np.unique(da_t['time.year'].values)
    ],
    dim='time',
)

# step 1, get hot days
da_t['time'] = da_90_cmp['time'] # make calendars the same
da_hd = xr.where(
    da_t>da_90_cmp,
    1,
    np.nan,
) 

# step 2, get hot periods
da_hd = da_hd.groupby(da_hd.isnull().cumsum(dim='time')).map(groupingfunc) # group between nans via cumsum of 1s, map vals as len of valid points in group 
da_hp = xr.where(da_hd>=3,1,np.nan) # keep only groups with at least 3 days (hot periods)
da_hp = da_t.where(da_hp==1)

# step 3, calculate magnitudes 
da_mgt = magnitude(
    da_hp,
    da_25,
    da_75,
)

# to verify this shit, I need to get a lat/lon pixel with valid hot period credentials, then check on a given year of data if my shit works

In [ ]:
# magnitude of hot period
def magnitude(
    da,
    da_25,
    da_75
):
    
    # run calc w.r.t. pic annual thresholds
    mgt = (da - da_25)/\
        (da_75 - da_25)
        
    # magnitudes for daily temps below 25% of PIC set to 0
    mgt = xr.where(da > da_25, mgt, np.nan)
    
    # sum magnitudes per hot period
    mgt = mgt.groupby(mgt.isnull().cumsum(dim='time')).sum(dim='time')
    
    # get highest per year
    mgt = mgt.groupby('time.year').max('time')
        
    return mgt